<a href="https://colab.research.google.com/github/noeramlpspta/online_retail/blob/main/Market_Basket_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation Data

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from scipy import stats

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Noer Amalia Portofolio's/Python/Online Retail Project/Online Retail Data.csv")
df

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN
...,...,...,...,...,...,...,...
461768,539991,21618,4 WILDFLOWER BOTANICAL CANDLES,1,2010-12-23 16:49:00,1.25,NaN
461769,539991,72741,GRAND CHOCOLATECANDLE,4,2010-12-23 16:49:00,1.45,NaN
461770,539992,21470,FLOWER VINE RAFFIA FOOD COVER,1,2010-12-23 17:41:00,3.75,NaN
461771,539992,22258,FELT FARM ANIMAL RABBIT,1,2010-12-23 17:41:00,1.25,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461773 entries, 0 to 461772
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      461773 non-null  object 
 1   product_code  461773 non-null  object 
 2   product_name  459055 non-null  object 
 3   quantity      461773 non-null  int64  
 4   order_date    461773 non-null  object 
 5   price         461773 non-null  float64
 6   customer_id   360853 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 24.7+ MB


# Cleaning Data

In [4]:
# Membuat salinan dari df untuk menghindari SettingWithCopyWarning
df_retail = df.copy()
# Membuat kolom order_date sebagai datetime dan menghapus baris dengan NaT
df_retail['order_date'] = pd.to_datetime(df_retail['order_date'], errors='coerce')
df_retail = df_retail.dropna(subset=['order_date'])
# Menghapus semua baris tanpa customer_id dan mengonversi customer_id menjadi string
df_retail = df_retail[~df_retail['customer_id'].isna()].copy()
df_retail['customer_id'] = df_retail['customer_id'].astype(str)
# Menghapus semua baris tanpa product_name dan membuat product_name huruf kecil
df_retail = df_retail[~df_retail['product_name'].isna()].copy()
df_retail['product_name'] = df_retail['product_name'].str.lower()
# Menghapus baris dengan 'test' di product_code atau product_name
df_retail = df_retail[
    (~df_retail['product_code'].str.lower().str.contains('test', na=False)) &
    (~df_retail['product_name'].str.contains('test', na=False))].copy()
# Menghapus baris dengan status cancelled (order_id yang diawali 'C')
df_retail = df_retail[~df_retail['order_id'].str.startswith('C')]
# Mengubah nilai quantity yang negatif menjadi positif
df_retail.loc[:, 'quantity'] = df_retail['quantity'].abs()
# Menghapus baris dengan price bernilai negatif
df_retail = df_retail[df_retail['price'] > 0].copy()
# Membuat kolom amount
df_retail['amount'] = df_retail['quantity'] * df_retail['price']
# Mengganti product_name berdasarkan product_code yang paling sering muncul
most_freq_product_name = (df_retail.groupby(['product_code', 'product_name'], as_index=False).agg(order_cnt=('order_id', 'nunique')).sort_values(['product_code', 'order_cnt'], ascending=[True, False]))
most_freq_product_name['rank'] = most_freq_product_name.groupby('product_code')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank'] == 1].drop(columns=['order_cnt', 'rank'])
df_retail = df_retail.merge(most_freq_product_name.rename(columns={'product_name': 'most_freq_product_name'}),
                            how='left', on='product_code')
df_retail['product_name'] = df_retail['most_freq_product_name']
df_retail = df_retail.drop(columns='most_freq_product_name')
# Menghapus outlier
df_retail = df_retail[(np.abs(stats.zscore(df_retail[['quantity', 'amount']])) < 3).all(axis=1)]
df_retail = df_retail.reset_index(drop=True)
df_retail


,order_id,product_code,product_name,quantity,order_date,price,customer_id,amount
0,493414,21844,red retrospot mug,36,2010-01-04 10:28:00,2.55,14590.0,91.80
1,493414,21533,retro spot large milk jug,12,2010-01-04 10:28:00,4.25,14590.0,51.00
2,493414,37508,new england ceramic cake server,2,2010-01-04 10:28:00,2.55,14590.0,5.10
3,493414,35001G,hand open shape gold,2,2010-01-04 10:28:00,4.25,14590.0,8.50
4,493414,21527,red retrospot traditional teapot,12,2010-01-04 10:28:00,6.95,14590.0,83.40
...,...,...,...,...,...,...,...,...
350087,539988,84380,set of 3 butterfly cookie cutters,1,2010-12-23 16:06:00,1.25,18116.0,1.25
350088,539988,84849D,hot baths soap holder,1,2010-12-23 16:06:00,1.69,18116.0,1.69
350089,539988,84849B,fairy soap soap holder,1,2010-12-23 16:06:00,1.69,18116.0,1.69
350090,539988,22854,cream sweetheart egg holder,2,2010-12-23 16:06:00,4.95,18116.0,9.90


In [5]:
df_retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350092 entries, 0 to 350091
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      350092 non-null  object        
 1   product_code  350092 non-null  object        
 2   product_name  350092 non-null  object        
 3   quantity      350092 non-null  int64         
 4   order_date    350092 non-null  datetime64[ns]
 5   price         350092 non-null  float64       
 6   customer_id   350092 non-null  object        
 7   amount        350092 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 21.4+ MB


# Menyiapkan Dataset Basket

Membuat Tabel Data Basket

In [6]:
# Membuat Tabel Dataset Basket
basket = pd.pivot_table(df_retail, index='order_id', columns='product_name', values='product_code', aggfunc='nunique', fill_value=0)
# Encode DataFrame basket dengan nilai True untuk semua nilai di atas 0 dan False untuk semua nilai 0
def encode(x):
    if x==0:
        return False
    if x>0:
        return True

basket_encode = basket.apply(lambda col: col.map(encode))
basket_encode

product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493433,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539981,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539982,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539985,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Mengambil Transaksi dengan banyaknya produk unik lebih dari 1

In [7]:
basket_filter = basket_encode[(basket_encode>0).sum(axis=1)>1]
basket_filter

product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493433,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539978,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539981,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539982,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Membuat list frequent itemset (kumpulan produk yang sering dibeli) dengan Apriori Algorithm

In [8]:
# pip install mlxtend
from mlxtend.frequent_patterns import apriori

frequent_itemset = apriori(basket_filter, min_support=.01, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
frequent_itemset['product_cnt'] = frequent_itemset['itemsets'].apply(lambda x: len(x))
frequent_itemset

,support,itemsets,product_cnt
0,0.177953,(white hanging heart t-light holder),1
1,0.099440,(regency cakestand 3 tier),1
2,0.096841,(jumbo bag red retrospot),1
3,0.079912,(pack of 72 retro spot cake cases),1
4,0.078512,(assorted colour bird ornament),1
...,...,...,...
1189,0.010064,"(key fob , back door , key fob , front door ,...",3
1190,0.010064,"(wrap,suki and friends)",1
1191,0.010064,(retrospot padded seat cushion),1
1192,0.010064,"(pack of 6 birdy gift tags, pack of 6 panneton...",2
